In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm_notebook
%matplotlib inline

In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm_notebook

def load_data(file):
    """Загрузка входных данных из файла"""
    with open(file, 'r') as f:
        input_data = json.load(f)
    couriers = {}
    orders = {}
    points = {}
    for depotData in input_data['depots']:
        points[depotData['point_id']] = {
            'location': [depotData['location_x'], depotData['location_y']],
            'timewindow': [0, 1439],
        }
    for courierData in input_data['couriers']:
        couriers[courierData['courier_id']] = {
            'location': [courierData['location_x'], courierData['location_y']],
            'time': 360,
        }
    for orderData in input_data['orders']:
        points[orderData['pickup_point_id']] = {
            'location': [orderData['pickup_location_x'], orderData['pickup_location_y']],
            'timewindow': [orderData['pickup_from'], orderData['pickup_to']],
            'order_time': {orderData['order_id']: orderData['pickup_from']}
        }
        points[orderData['dropoff_point_id']] = {
            'location': [orderData['dropoff_location_x'], orderData['dropoff_location_y']],
            'timewindow': [orderData['dropoff_from'], orderData['dropoff_to']],
        }
        orders[orderData['order_id']] = orderData
    return couriers, orders, points

In [91]:
import json
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm_notebook

def load_data(file):
    """Загрузка входных данных из файла"""
    with open(file, 'r') as f:
        input_data = json.load(f)
    couriers = {}
    orders = {}
    points = {}
    for depotData in input_data['depots']:
        points[depotData['point_id']] = {
            'location': [depotData['location_x'], depotData['location_y']],
            'timewindow': [0, 1439],
        }
    for courierData in input_data['couriers']:
        couriers[courierData['courier_id']] = {
            'location': [courierData['location_x'], courierData['location_y']],
            'time': 360,
        }
    for orderData in input_data['orders']:
        points[orderData['pickup_point_id']] = {
            'location': [orderData['pickup_location_x'], orderData['pickup_location_y']],
            'timewindow': [orderData['pickup_from'], orderData['pickup_to']],
            'order_time': {orderData['order_id']: orderData['pickup_from']}
        }
        points[orderData['dropoff_point_id']] = {
            'location': [orderData['dropoff_location_x'], orderData['dropoff_location_y']],
            'timewindow': [orderData['dropoff_from'], orderData['dropoff_to']],
        }
        orders[orderData['order_id']] = orderData
    return couriers, orders, points

def get_travel_duration_minutes(location1, location2):
    """Время перемещения курьера от точки location1 до точки location2 вминутах"""
    distance = abs(location1[0] - location2[0]) + abs(location1[1] - location2[1])
    return 10 + distance

def get_current_courier_reward(time, payment):
    work_payment = time * 2
    profit = orders_payment - work_payment
    return profit

def delete_element(array, idx):
    result = []
    for i in range(len(array)):
        if not(i == idx):
            result.append(array[i])
    return result

def main():
    payment = 0
    data = load_data("data/contest_input.json")
    reward = 0
    stop_flag = 0
    couriers_start_position = [data[0][i+1]['location'] for i in range(len(data[0]))]
    points_pickup = np.array([[data[1][i+10001]['pickup_location_x'],
                               data[1][i+10001]['pickup_location_y']] for i in range(len(data[1]))])
    time_pickup = np.array([[data[1][10001+i]['pickup_from'],
                             data[1][10001+i]['pickup_to']] for i in range(len(data[1]))])
    time_dropoff = np.array([[data[1][10001+i]['dropoff_from'],
                              data[1][10001+i]['dropoff_to']] for i in range(len(data[1]))])    
    points_dropoff = np.array([[data[1][i+10001]['dropoff_location_x'],
                                data[1][i+10001]['dropoff_location_y']] for i in range(len(data[1]))])
    payments = np.array([data[1][10001+i]['payment'] for i in range(len(data[1]))])
    pickup_id = np.array([data[1][10001+i]['pickup_point_id'] for i in range(len(data[1]))])
    dropoff_id = np.array([data[1][10001+i]['dropoff_point_id'] for i in range(len(data[1]))])
    cnt = 0
    cnt_courier = 0
    routes = []
    for courier in tqdm_notebook(couriers_start_position):
        if stop_flag == 1:
            break 
        print('courier {} begins route at {}'.format(cnt_courier, courier))
        cnt_courier += 1
        tmp_x = [courier[0]]
        tmp_y = [courier[1]]
        print('processing courier {}..\n'.format(cnt))
        cnt += 1
        courier_position = courier
        current_time = 360
        distances = []
        routes = []
        # select avaliable pickup points
        while current_time < 1400:
            distances = []
            avaliable_pickups = []
            print("current time {}. Passing order {}\n".format(current_time, cnt))
            cnt += 1
            tmp_route = []
            for j in range(len(points_pickup)):
                tmp_route.append( {"courier_id": str(cnt_courier),
                                  "action":"pickup", "order_id":str(10001+j),
                                  "point_id":str(pickup_id[j])})
                tmp_route.append( {"courier_id": str(cnt_courier),
                                  "action":"dropoff", "order_id":str(10001+j),
                                  "point_id":str(dropoff_id[j])})
                # calculate route_time for each point from current courier position to dropoff
                route_time = get_travel_duration_minutes(points_pickup[j],points_dropoff[j]) + get_travel_duration_minutes(points_pickup[j],
                                                                                                 courier_position)
                if current_time in time_pickup[j] and current_time + route_time:
                    avaliable_pickups.append(points_pickup[j])       

            # calculate Manhattan distance to closest avaliable points
            min_dist = np.inf
            if len(avaliable_pickups) == 0:
                break
                print('сука')
            for i in range(len(avaliable_pickups)):
                dist = get_travel_duration_minutes(courier, avaliable_pickups[i])
                if not(dist == 0):
                    tmp = get_travel_duration_minutes(courier, avaliable_pickups[i])
                    if tmp < min_dist:
                        min_dist = tmp
                        current_order = i

            # delete visited location

            print('the most far point: {}'.format(min_dist))
            courier_position = points_dropoff[j]
            print('send courier on {}'.format(courier_position))
            tmp_x.append(courier_position[0])
            tmp_y.append(courier_position[1])

            current_time += route_time
            payment = payments[current_order] 
            print('Payment: {}, salary: {}'.format(payments[current_order], route_time))

            #deleting visited elements
            points_dropoff = delete_element(points_dropoff,current_order)
            time_pickup = delete_element(time_pickup,current_order)
            points_pickup = delete_element(points_pickup, current_order)
            time_dropoff = delete_element(time_dropoff, current_order)
            payments = delete_element(payments, current_order)

            print("{} points left".format(len(points_pickup)))
        reward += payment   
        print(reward)
        print("-"*80)
        routes.append(tmp_route)
    print('total reward: {}'.format(reward - (current_time-360)*2))
    routes = routes[0]
    routes_dict = dict({})
    for i in range(len(routes)):
        routes_dict.update({str(i):routes[i]})
    data=json.dumps(routes_dict)
    with open('output.json', 'w') as outfile:
        json.dump(data, outfile)
if __name__ == '__main__':
    main()

courier 0 begins route at [111, 5]
processing courier 0..

current time 360. Passing order 1

the most far point: 18
send courier on [135  51]
Payment: 268, salary: 434
7302 points left
current time 794. Passing order 2

268
--------------------------------------------------------------------------------
courier 1 begins route at [260, 112]
processing courier 3..

current time 360. Passing order 4

the most far point: 11
send courier on [135  51]
Payment: 700, salary: 212
7301 points left
current time 572. Passing order 5

the most far point: 40
send courier on [135  51]
Payment: 302, salary: 364
7300 points left
current time 936. Passing order 6

570
--------------------------------------------------------------------------------
courier 2 begins route at [207, 66]
processing courier 7..

current time 360. Passing order 8

the most far point: 17
send courier on [135  51]
Payment: 302, salary: 277
7299 points left
current time 637. Passing order 9

the most far point: 93
send courier o

the most far point: 13
send courier on [135  51]
Payment: 250, salary: 264
7267 points left
current time 624. Passing order 79

the most far point: 32
send courier on [135  51]
Payment: 650, salary: 364
7266 points left
current time 988. Passing order 80

the most far point: 108
send courier on [135  51]
Payment: 310, salary: 364
7265 points left
current time 1352. Passing order 81

8313
--------------------------------------------------------------------------------
courier 22 begins route at [84, 155]
processing courier 82..

current time 360. Passing order 83

the most far point: 17
send courier on [135  51]
Payment: 290, salary: 391
7264 points left
current time 751. Passing order 84

the most far point: 50
send courier on [135  51]
Payment: 244, salary: 364
7263 points left
current time 1115. Passing order 85

8557
--------------------------------------------------------------------------------
courier 23 begins route at [222, 137]
processing courier 86..

current time 360. Passin

15896
--------------------------------------------------------------------------------
courier 43 begins route at [220, 130]
processing courier 158..

current time 360. Passing order 159

the most far point: 13
send courier on [135  51]
Payment: 603, salary: 230
7230 points left
current time 590. Passing order 160

the most far point: 22
send courier on [135  51]
Payment: 321, salary: 364
7229 points left
current time 954. Passing order 161

the most far point: 41
send courier on [135  51]
Payment: 342, salary: 364
7228 points left
current time 1318. Passing order 162

16238
--------------------------------------------------------------------------------
courier 44 begins route at [288, 106]
processing courier 163..

current time 360. Passing order 164

the most far point: 13
send courier on [135  51]
Payment: 284, salary: 246
7227 points left
current time 606. Passing order 165

the most far point: 54
send courier on [135  51]
Payment: 242, salary: 364
7226 points left
current time 97

the most far point: 13
send courier on [135  51]
Payment: 250, salary: 312
7197 points left
current time 672. Passing order 237

23542
--------------------------------------------------------------------------------
courier 66 begins route at [58, 211]
processing courier 238..

current time 360. Passing order 239

the most far point: 30
send courier on [135  51]
Payment: 254, salary: 473
7196 points left
current time 833. Passing order 240

23796
--------------------------------------------------------------------------------
courier 67 begins route at [147, 200]
processing courier 241..

current time 360. Passing order 242

the most far point: 15
send courier on [135  51]
Payment: 504, salary: 373
7195 points left
current time 733. Passing order 243

24300
--------------------------------------------------------------------------------
courier 68 begins route at [361, 121]
processing courier 244..

current time 360. Passing order 245

the most far point: 16
send courier on [135  51]
P

the most far point: 51
send courier on [135  51]
Payment: 545, salary: 364
7161 points left
current time 997. Passing order 311

30311
--------------------------------------------------------------------------------
courier 85 begins route at [190, 127]
processing courier 312..

current time 360. Passing order 313

the most far point: 18
send courier on [135  51]
Payment: 604, salary: 257
7160 points left
current time 617. Passing order 314

the most far point: 53
send courier on [135  51]
Payment: 271, salary: 364
7159 points left
current time 981. Passing order 315

30582
--------------------------------------------------------------------------------
courier 86 begins route at [145, 123]
processing courier 316..

current time 360. Passing order 317

the most far point: 12
send courier on [135  51]
Payment: 250, salary: 298
7158 points left
current time 658. Passing order 318

the most far point: 25
send courier on [135  51]
Payment: 483, salary: 364
7157 points left
current time 102

...

the most far point: 12
send courier on [135  51]
Payment: 423, salary: 431
6946 points left
current time 791. Passing order 764

75654
--------------------------------------------------------------------------------
courier 204 begins route at [162, 158]
processing courier 765..

current time 360. Passing order 766

the most far point: 15
send courier on [135  51]
Payment: 309, salary: 316
6945 points left
current time 676. Passing order 767

75963
--------------------------------------------------------------------------------
courier 205 begins route at [201, 138]
processing courier 768..

current time 360. Passing order 769

the most far point: 12
send courier on [135  51]
Payment: 290, salary: 257
6944 points left
current time 617. Passing order 770

the most far point: 62
send courier on [135  51]
Payment: 352, salary: 364
6943 points left
current time 981. Passing order 771

76315
--------------------------------------------------------------------------------
courier 206 begins 

83131
--------------------------------------------------------------------------------
courier 226 begins route at [241, 70]
processing courier 842..

current time 360. Passing order 843

the most far point: 11
send courier on [135  51]
Payment: 291, salary: 239
6912 points left
current time 599. Passing order 844

the most far point: 85
send courier on [135  51]
Payment: 231, salary: 364
6911 points left
current time 963. Passing order 845

83362
--------------------------------------------------------------------------------
courier 227 begins route at [229, 143]
processing courier 846..

current time 360. Passing order 847

the most far point: 14
send courier on [135  51]
Payment: 190, salary: 234
6910 points left
current time 594. Passing order 848

the most far point: 27
send courier on [135  51]
Payment: 440, salary: 364
6909 points left
current time 958. Passing order 849

83802
--------------------------------------------------------------------------------
courier 228 begins r

the most far point: 22
send courier on [135  51]
Payment: 356, salary: 427
6877 points left
current time 787. Passing order 919

91235
--------------------------------------------------------------------------------
courier 247 begins route at [167, 94]
processing courier 920..

current time 360. Passing order 921

the most far point: 20
send courier on [135  51]
Payment: 364, salary: 289
6876 points left
current time 649. Passing order 922

the most far point: 51
send courier on [135  51]
Payment: 424, salary: 364
6875 points left
current time 1013. Passing order 923

91659
--------------------------------------------------------------------------------
courier 248 begins route at [189, 144]
processing courier 924..

current time 360. Passing order 925

the most far point: 14
send courier on [135  51]
Payment: 610, salary: 275
6874 points left
current time 635. Passing order 926

the most far point: 33
send courier on [135  51]
Payment: 467, salary: 364
6873 points left
current time 9

the most far point: 48
send courier on [135  51]
Payment: 422, salary: 364
6841 points left
current time 951. Passing order 995

99428
--------------------------------------------------------------------------------
courier 267 begins route at [69, 9]
processing courier 996..

current time 360. Passing order 997

the most far point: 36
send courier on [135  51]
Payment: 397, salary: 472
6840 points left
current time 832. Passing order 998

99825
--------------------------------------------------------------------------------
courier 268 begins route at [137, 134]
processing courier 999..

current time 360. Passing order 1000

the most far point: 13
send courier on [135  51]
Payment: 373, salary: 317
6839 points left
current time 677. Passing order 1001

100198
--------------------------------------------------------------------------------
courier 269 begins route at [195, 116]
processing courier 1002..

current time 360. Passing order 1003

the most far point: 13
send courier on [135 

the most far point: 88
send courier on [135  51]
Payment: 662, salary: 364
6805 points left
current time 1352. Passing order 1071

107506
--------------------------------------------------------------------------------
courier 287 begins route at [191, 161]
processing courier 1072..

current time 360. Passing order 1073

the most far point: 13
send courier on [135  51]
Payment: 315, salary: 290
6804 points left
current time 650. Passing order 1074

the most far point: 59
send courier on [135  51]
Payment: 389, salary: 364
6803 points left
current time 1014. Passing order 1075

the most far point: 203
send courier on [135  51]
Payment: 474, salary: 364
6802 points left
current time 1378. Passing order 1076

107980
--------------------------------------------------------------------------------
courier 288 begins route at [81, 20]
processing courier 1077..

current time 360. Passing order 1078

the most far point: 13
send courier on [135  51]
Payment: 254, salary: 449
6801 points left
cu

In [81]:
routes = routes[0]

In [82]:
routes

[{'courier_id': '300',
  'action': 'pickup',
  'order_id': '10001',
  'point_id': '40001'},
 {'courier_id': '300',
  'action': 'dropoff',
  'order_id': '10001',
  'point_id': '60001'},
 {'courier_id': '300',
  'action': 'pickup',
  'order_id': '10002',
  'point_id': '40002'},
 {'courier_id': '300',
  'action': 'dropoff',
  'order_id': '10002',
  'point_id': '60002'},
 {'courier_id': '300',
  'action': 'pickup',
  'order_id': '10003',
  'point_id': '40003'},
 {'courier_id': '300',
  'action': 'dropoff',
  'order_id': '10003',
  'point_id': '60003'},
 {'courier_id': '300',
  'action': 'pickup',
  'order_id': '10004',
  'point_id': '40004'},
 {'courier_id': '300',
  'action': 'dropoff',
  'order_id': '10004',
  'point_id': '60004'},
 {'courier_id': '300',
  'action': 'pickup',
  'order_id': '10005',
  'point_id': '40005'},
 {'courier_id': '300',
  'action': 'dropoff',
  'order_id': '10005',
  'point_id': '60005'},
 {'courier_id': '300',
  'action': 'pickup',
  'order_id': '10006',
  'poin

In [83]:
routes_dict = dict({})

In [84]:
for i in range(len(routes)):
    routes_dict.update({str(i):routes[i]})

In [86]:
data=json.dumps(routes_dict)

In [87]:
with open('data.txt', 'w') as outfile:
    json.dump(data, outfile)